## Настройка

In [1]:
#r "nuget: Plotly.NET.Interactive, 4.1.0"
#load "Imports.fsx"
#r "nuget: TorchSharp-cuda-windows"

Installed Packages Plotly.NET.Interactive, 4.1.0 TorchSharp-cuda-windows, 0.100.3

Loading extensions from `C:\Users\Asus\.nuget\packages\skiasharp\2.88.3\interactive-extensions\dotnet\SkiaSharp.DotNet.Interactive.dll`

Loading extensions from `C:\Users\Asus\.nuget\packages\plotly.net.interactive\4.1.0\interactive-extensions\dotnet\Plotly.NET.Interactive.dll`

In [2]:
open System
open Plotly.NET
open Plotly.NET.TraceObjects
open Plotly.NET.LayoutObjects
open System.Drawing
open Microsoft.DotNet.Interactive.Formatting
open Investor.Core.FSharp
open Investor.Core.FSharp.Indicators
open Investor.Core.FSharp.Strategies.TradeStrategy
open Investor.Core.Trading
open Investor.Core.Strategies
open Investor.Core.Entities
open Investor.Interactive
open Investor.Interactive.Training
open Investor.Optimizations.Training.Online
open Investor.Interactive.Training.Online
open Investor.Interactive.DrawingScope
open Investor.Optimizations.Labeling
open Investor.Core.FSharp.Strategies.TradeDealStrategy
open System.Text.Json
open Skender.Stock.Indicators
open Investor.Optimizations.Labeling.TrendLabel
open Investor.Core.FSharp.Point
open Investor.Optimizations.Utils
open TorchSharp
open TorchSharp.Modules

In [3]:
// Конфигурируем форматирование
Interactive.addFormatters() |> ignore

// Методы для отрисовки графика
module LabelChart =

    let toChart name color points =
        Chart.Point(
            points |> Seq.map(fun (x: IPoint) -> x.OccuredOn),
            points |> Seq.map(fun (x: IPoint) -> x.Value)
        )
        |> Chart.withTraceInfo (Name = name)
        |> Chart.withMarkerStyle (Size = 10, Color = color)

    let toLineChart name color points =
        Chart.Line(
            points |> Seq.map(fun (x: IPoint) -> x.OccuredOn),
            points |> Seq.map(fun (x: IPoint) -> x.Value)
        )
        |> Chart.withTraceInfo (Name = name)
        |> Chart.withMarkerStyle (Size = 10, Color = color)

    let trendToCandlePoints (candles: Candle seq) = 
        TrendLabel.toSeqPoints candles (fun x -> float x.Close)

    let draw (candles: Candle seq) (orderLables: OrderLabel list) (trend: TrendLabel list) = 
        let buyes = 
            orderLables |> Seq.map(fun x -> Point.create x.BuyCandle.OccuredOn (float x.BuyCandle.Close))

        let sells = 
            orderLables |> Seq.map(fun x -> Point.create x.SellCandle.OccuredOn (float x.SellCandle.Close))

        [
            [
                (candles |> Seq.toList |> Charts.candlestick)
                buyes |> toChart "Buy" (Color.fromHex "#C70039")
                sells |> toChart "Sell" (Color.fromHex "#343090")
                trend |> (trendToCandlePoints candles) |> toLineChart "trend" (Color.fromHex "#2A60A0")
            ] |> Chart.combine
        ] 
        |> Chart.SingleStack(Pattern = StyleParam.LayoutGridPattern.Coupled)
        |> Chart.withSize(1500, 1000)

## Работа с данными

In [4]:
// Конфигурируем источник данных
// Даты c, ценную бумагу, интервал

let scopeFactory = TrainingScopeFactory(
                        Options.Create
                            (new DateTime(2022, 3, 28))
                            (new DateTime(2022, 5, 15))
                            "BBG004730N88" 
                            CandleSize.FifteenMinutes
                    )

// Загружаем данные
scopeFactory.WarmUp()
    |> Async.AwaitTask
    |> Async.RunSynchronously

In [5]:
scopeFactory.Candles 
|> Interactive.showSeq 10

SecurityId,Volume,Close,High,Low,Open,Size,OccuredOn
BBG004730N88,800480,"125,99","131,47","125,01","130,6",FifteenMinutes,2022-03-28 07:00:00Z
BBG004730N88,445971,"127,66","128,3","125,13","125,97",FifteenMinutes,2022-03-28 07:15:00Z
BBG004730N88,381323,"126,1","128,4","125,4","127,68",FifteenMinutes,2022-03-28 07:30:00Z
BBG004730N88,169117,"126,52",127,"125,97","126,1",FifteenMinutes,2022-03-28 07:45:00Z
BBG004730N88,136321,"126,64","126,95","126,45","126,52",FifteenMinutes,2022-03-28 08:00:00Z
BBG004730N88,267263,"126,4",127,"126,03","126,74",FifteenMinutes,2022-03-28 08:15:00Z
BBG004730N88,115840,"126,2","126,89","126,2","126,4",FifteenMinutes,2022-03-28 08:30:00Z
BBG004730N88,75242,"126,48","126,57","126,11","126,28",FifteenMinutes,2022-03-28 08:45:00Z
BBG004730N88,237426,"126,82","127,8","126,11","126,48",FifteenMinutes,2022-03-28 09:00:00Z
BBG004730N88,81496,"126,47",127,"126,4","126,82",FifteenMinutes,2022-03-28 09:15:00Z


In [6]:
// Размечаем по сделкам
let orderLables = 
    scopeFactory.Candles
        |> Seq.toList
        |> OrderLabel.find 
            FrameBuilder.defaultOrderLabelOptions
        |> Seq.toList

orderLables
|> Interactive.showSeq 4

Total: 23 BuyCandle SellCandle Buy Sell Profit CommissionProfit ProfitPercent Commission Length Interval 
 
 
 SecurityId 
 BBG004730N88 
 
 
 Volume 
 74208 
 
 
 Close 
 125.000000 
 
 
 High 
 125.000000 
 
 
 Low 
 125.000000 
 
 
 Open 
 125.000000 
 
 
 
 
 
 SecurityId 
 BBG004730N88 
 
 
 Volume 
 861698 
 
 
 Close 
 136.920000 
 
 
 High 
 137.570000 
 
 
 Low 
 134.100000 
 
 
 Open 
 134.300000 
 
 
 125 136,92 11.92 11.841424 0.0953599999999999 0.078576 22:00:00 28.03.2022 10:45:00 - 29.03.2022 8:45:00 
 
 
 SecurityId 
 BBG004730N88 
 
 
 Volume 
 445229 
 
 
 Close 
 126.350000 
 
 
 High 
 128.000000 
 
 
 Low 
 125.820000 
 
 
 Open 
 127.520000 
 
 
 
 
 
 SecurityId 
 BBG004730N88 
 
 
 Volume 
 527076 
 
 
 Close 
 146.300000 
 
 
 High 
 147.410000 
 
 
 Low 
 145.000000 
 
 
 Open 
 146.320000 
 
 
 126,35 146,3 19.95 19.868205 0.15789473684210542 0.08179499999999999 2.01:45:00 29.03.2022 10:00:00 - 31.03.2022 11:45:00 
 
 
 SecurityId 
 BBG004730N88 
 
 
 Volume 
 194676 
 
 
 Close 
 141.510000 
 
 
 High 
 142.000000 
 
 
 Low 
 140.460000 
 
 
 Open 
 141.650000 
 
 
 
 
 
 SecurityId 
 BBG004730N88 
 
 
 Volume 
 950531 
 
 
 Close 
 154.170000 
 
 
 High 
 154.480000 
 
 
 Low 
 151.940000 
 
 
 Open 
 151.980000 
 
 
 141,51 154,17 12.66 12.571296 0.08946364214543141 0.08870399999999998 19:00:00 31.03.2022 13:00:00 - 01.04.2022 8:00:00 
 
 
 SecurityId 
 BBG004730N88 
 
 
 Volume 
 424109 
 
 
 Close 
 150.650000 
 
 
 High 
 154.000000 
 
 
 Low 
 150.610000 
 
 
 Open 
 152.500000 
 
 
 
 
 
 SecurityId 
 BBG004730N88 
 
 
 Volume 
 230825 
 
 
 Close 
 155.250000 
 
 
 High 
 155.340000 
 
 
 Low 
 154.010000 
 
 
 Open 
 154.010000 
 
 
 150,65 155,25 4.6 4.508229999999999 0.03053435114503813 0.09176999999999999 06:00:00 01.04.2022 8:30:00 - 01.04.2022 14:30:00

In [7]:
// Размечаем тренд
let trendLables = 
    scopeFactory.Candles 
    |> Seq.toList
    |> TrendLabel.find 
        (fun x -> x.Close |> float) 
        FrameBuilder.defaultTrendLabelOptions
    |> Seq.toList

trendLables
|> Interactive.showSeq 10

StartDate,EndDate,Start,End,LabelType,Length,Interval
2022-03-28 10:00:00Z,2022-03-28 10:00:00Z,0.1,0.1,Flat,00:00:00,28.03.2022 10:00:00 - 28.03.2022 10:00:00
2022-03-28 10:15:00Z,2022-03-28 10:15:00Z,0.4,0.4,Rally,00:00:00,28.03.2022 10:15:00 - 28.03.2022 10:15:00
2022-03-28 10:30:00Z,2022-03-29 08:00:00Z,0.5,0.8,BigRally,21:30:00,28.03.2022 10:30:00 - 29.03.2022 8:00:00
2022-03-29 08:00:00Z,2022-03-29 08:00:00Z,0.8,0.8,BigRally,00:00:00,29.03.2022 8:00:00 - 29.03.2022 8:00:00
2022-03-29 08:15:00Z,2022-03-29 08:15:00Z,0.2,0.2,Rally,00:00:00,29.03.2022 8:15:00 - 29.03.2022 8:15:00
2022-03-29 08:30:00Z,2022-03-29 08:45:00Z,-0.3,-0.3,Correction,00:15:00,29.03.2022 8:30:00 - 29.03.2022 8:45:00
2022-03-29 08:45:00Z,2022-03-29 08:45:00Z,-0.3,-0.3,Correction,00:00:00,29.03.2022 8:45:00 - 29.03.2022 8:45:00
2022-03-29 09:00:00Z,2022-03-29 09:30:00Z,-0.2,-0.1,Flat,00:30:00,29.03.2022 9:00:00 - 29.03.2022 9:30:00
2022-03-29 09:30:00Z,2022-03-29 09:30:00Z,-0.1,-0.1,Flat,00:00:00,29.03.2022 9:30:00 - 29.03.2022 9:30:00
2022-03-29 09:45:00Z,2022-03-29 09:45:00Z,-0.5,-0.5,Correction,00:00:00,29.03.2022 9:45:00 - 29.03.2022 9:45:00


In [8]:
// Рисуем размеченное
LabelChart.draw
    scopeFactory.Candles
    orderLables
    trendLables

<!-- Plotly chart will be drawn inside this DIV -->

![1](newplot.png)

![2](newplot2.png)

In [16]:
// Некоторая обобщенная функция которая формирует 
// табличку в соответствие с графиком
// Вешает на каждую свечу один из лейблов
//
// BigRally
// Buy
// Correction
// Flat
// Rally
// Sell
// BigCorrection

let frames =
        scopeFactory.Candles
        |> Seq.toList
        |> FrameBuilder.getLabeledFrames
               FrameBuilder.defaultOrderLabelOptions
               FrameBuilder.defaultTrendLabelOptions
        |> Seq.toList

frames 
|> Interactive.showSeq 5

Total: 804 Candle LabelType 
 
 
 SecurityId 
 BBG004730N88 
 
 
 Volume 
 85124 
 
 
 Close 
 125.810000 
 
 
 High 
 126.020000 
 
 
 Low 
 125.600000 
 
 
 Open 
 125.700000 
 
 
 
 BigRally 
 
 
 
 SecurityId 
 BBG004730N88 
 
 
 Volume 
 74208 
 
 
 Close 
 125.000000 
 
 
 High 
 125.000000 
 
 
 Low 
 125.000000 
 
 
 Open 
 125.000000 
 
 
 
 Buy 
 
 
 
 SecurityId 
 BBG004730N88 
 
 
 Volume 
 481817 
 
 
 Close 
 126.950000 
 
 
 High 
 127.000000 
 
 
 Low 
 122.000000 
 
 
 Open 
 126.190000 
 
 
 
 BigRally 
 
 
 
 SecurityId 
 BBG004730N88 
 
 
 Volume 
 916633 
 
 
 Close 
 130.300000 
 
 
 High 
 130.440000 
 
 
 Low 
 126.200000 
 
 
 Open 
 126.950000 
 
 
 
 BigRally 
 
 
 
 SecurityId 
 BBG004730N88 
 
 
 Volume 
 827601 
 
 
 Close 
 130.620000 
 
 
 High 
 132.790000 
 
 
 Low 
 129.040000 
 
 
 Open 
 130.300000 
 
 
 
 BigRally

In [10]:
// Простотр всех лейблов
frames 
|> Seq.distinctBy(fun x -> x.LabelType)
|> Seq.map (fun x -> {| LabelType = x.LabelType |})
|> Interactive.showSeq 10

LabelType
BigRally
Buy
Correction
Flat
Rally
Sell
BigCorrection


## Нейронка и Обучение

In [11]:
// Вспомогательные методы для обучения
module TrainContext =
    type TrainContext =
        { 
            criterion: CrossEntropyLoss; 
            model: torch.nn.Module<torch.Tensor, torch.Tensor>; 
            optimizer: Adam 
        }
        
    type TrainResult =
        { 
            Epoch: int; 
            TrainLoss: float; 
            ValidateLoss: float 
        }
    
    let getTrainContext (model: torch.nn.Module<torch.Tensor, torch.Tensor>) =
        {
            model = model
            optimizer = torch.optim.Adam(model.parameters(), lr = 2e-2)
            criterion = torch.nn.CrossEntropyLoss()
        }
            
    // Тренировка
    let trainBatch
        iterator
        context =

        let mutable totalLoss = 0.
        context.model.train()
        
        for data, labels in iterator do
            
            context.optimizer.zero_grad()
            let predict = context.model.forward(data)
            let loss = context.criterion.forward(predict, labels)
            loss.backward()
            context.optimizer.step() |> ignore
            totalLoss <- totalLoss + loss.item<float>() * (data.size(0) |> float)
            
        totalLoss <- totalLoss / (iterator |> Seq.length |> float)
        
        totalLoss
        
    // Валидация
    let validateBatch
        iterator
        context =

        let mutable totalLoss = 0.
        context.model.eval()
        
        for data, labels in iterator do
            
            let predict = context.model.forward(data)
            let loss = context.criterion.forward(predict, labels)
            totalLoss <- totalLoss + loss.item<float>() * (data.size(0) |> float)
            
        totalLoss <- totalLoss / (iterator |> Seq.length |> float)
        
        totalLoss

    // Полный цикл обучения
    let train iterator epochs context =
        seq {
            for i in 0 .. epochs - 1 do
            
                // Тренировка
                let trainLoss = context |> trainBatch iterator

                // Валидация
                // Пока что тут валидируем теми же самыми данными 
                // Которыми тренируем
                let validateLoss = context |> validateBatch iterator

                yield { Epoch = i; TrainLoss = trainLoss; ValidateLoss = validateLoss }
        }

In [12]:
// Преобразования в тензоры
module CandleQuote =

    // Преобразование свечи в тензон
    let toTensor (candle : CandleQuote) =
            torch.tensor(
                [
                 candle.Close
                 candle.High
                 candle.Low
                 candle.Open
                 candle.Volume
                ] |> Seq.toArray).reshape(1, 5).float()

module LabelType =

    // Маппинг на значений на значение тензора
    let trendLabelTypeMapping = [
        (FrameBuilder.LabelType.Buy, 1.)
        (FrameBuilder.LabelType.BigRally, 0.9)
        (FrameBuilder.LabelType.Rally, 0.7)
        (FrameBuilder.LabelType.Flat, 0.5)
        (FrameBuilder.LabelType.Correction, 0.3)
        (FrameBuilder.LabelType.BigCorrection, 0.1)
        (FrameBuilder.LabelType.Sell, 0)
    ]
    
    let toTensor (labelType: FrameBuilder.LabelType) =
        
        let value = 
            trendLabelTypeMapping
            |> Seq.where (fun (x, _) -> x = labelType)
            |> Seq.head
            |> snd
        
        torch.tensor([ value ] |> Seq.toArray).float()

In [17]:
// Размер окна
let predictLength = 100

let tensorData = 
    frames
        |> Seq.windowed predictLength
        |> Seq.map(fun window ->
            
            let data =
                window
                |> Seq.map(fun x -> x.Candle)
                |> Seq.map CandleQuote.fromCandle
                |> Seq.toList
                |> CandleQuote.scale 0 1 // Масштабируем окно от 0 до 1
                |> Seq.toArray
                
            let label =
                window
                |> Seq.last
                |> (fun x -> x.LabelType)
            
            (data, label)
        )
        
// Данные из которых будем формировать тензоры
// в данном списке находится кортеж (свячи, лейбл)
// свечи отмасштабированы от 0 до 1
tensorData 
|> Interactive.showSeq 1

Total: 705 Item1 Item2 index value 0 
 
 
 Volume 
 0.046860 
 
 
 Close 
 0.020449 
 
 
 High 
 0.025500 
 
 
 Low 
 0.085714 
 
 
 Open 
 0.017509 
 
 
 1 
 
 
 Volume 
 0.038971 
 
 
 Close 
 0.000000 
 
 
 High 
 0.000000 
 
 
 Low 
 0.071429 
 
 
 Open 
 0.000000 
 
 
 2 
 
 
 Volume 
 0.333552 
 
 
 Close 
 0.049230 
 
 
 High 
 0.050000 
 
 
 Low 
 0.000000 
 
 
 Open 
 0.029765 
 
 
 3 
 
 
 Volume 
 0.647796 
 
 
 Close 
 0.133805 
 
 
 High 
 0.136000 
 
 
 Low 
 0.100000 
 
 
 Open 
 0.048774 
 
 
 4 
 
 
 Volume 
 0.583452 
 
 
 Close 
 0.141883 
 
 
 High 
 0.194750 
 
 
 Low 
 0.167619 
 
 
 Open 
 0.132566 
 
 
 5 
 
 
 Volume 
 0.256539 
 
 
 Close 
 0.175208 
 
 
 High 
 0.180000 
 
 
 Low 
 0.197381 
 
 
 Open 
 0.141071 
 
 
 6 
 
 
 Volume 
 0.210174 
 
 
 Close 
 0.237314 
 
 
 High 
 0.247500 
 
 
 Low 
 0.250714 
 
 
 Open 
 0.190345 
 
 
 7 
 
 
 Volume 
 0.231908 
 
 
 Close 
 0.273668 
 
 
 High 
 0.299750 
 
 
 Low 
 0.295714 
 
 
 Open 
 0.297649 
 
 
 8 
 
 
 Volume 
 0.163386 
 
 
 Close 
 0.202979 
 
 
 High 
 0.271250 
 
 
 Low 
 0.262381 
 
 
 Open 
 0.271136 
 
 
 9 
 
 
 Volume 
 0.307110 
 
 
 Close 
 0.034082 
 
 
 High 
 0.075000 
 
 
 Low 
 0.090952 
 
 
 Open 
 0.063032 
 
 
 10 
 
 
 Volume 
 0.000000 
 
 
 Close 
 0.095178 
 
 
 High 
 0.094250 
 
 
 Low 
 0.161190 
 
 
 Open 
 0.094297 
 
 
 11 
 
 
 Volume 
 0.494290 
 
 
 Close 
 0.240091 
 
 
 High 
 0.335000 
 
 
 Low 
 0.220714 
 
 
 Open 
 0.287644 
 
 
 12 
 
 
 Volume 
 0.203209 
 
 
 Close 
 0.234284 
 
 
 High 
 0.237750 
 
 
 Low 
 0.245476 
 
 
 Open 
 0.237619 
 
 
 13 
 
 
 Volume 
 0.147602 
 
 
 Close 
 0.204494 
 
 
 High 
 0.237500 
 
 
 Low 
 0.223333 
 
 
 Open 
 0.232616 
 
 
 14 
 
 
 Volume 
 0.127994 
 
 
 Close 
 0.214087 
 
 
 High 
 0.239250 
 
 
 Low 
 0.261905 
 
 
 Open 
 0.205103 
 
 
 15 
 
 
 Volume 
 0.084511 
 
 
 Close 
 0.186064 
 
 
 High 
 0.212250 
 
 
 Low 
 0.240476 
 
 
 Open 
 0.207854 
 
 
 16 
 
 
 Volume 
 0.059753 
 
 
 Close 
 0.188841 
 
 
 High 
 0.187500 
 
 
 Low 
 0.241190 
 
 
 Open 
 0.184342 
 
 
 17 
 
 
 Volume 
 0.146374 
 
 
 Close 
 0.173946 
 
 
 High 
 0.200000 
 
 
 Low 
 0.221429 
 
 
 Open 
 0.187594 
 
 
 18 
 
 
 Volume 
 0.085971 
 
 
 Close 
 0.178995 
 
 
 High 
 0.182500 
 
 
 Low 
 0.216905 
 
 
 Open 
 0.179340 
 
 
 19 
 
 
 Volume 
 0.055884 
 
 
 Close 
 0.176471 
 
 
 High 
 0.185500 
 
 
 Low 
 0.234524 
 
 
 Open 
 0.174587 
 
 
 (80 more) 
 Rally

In [18]:
// Модель
// Тут больше всего вопросов, потому как 
// Нет понимания по каким принципам ее необходимо строить
// Нет понимания какой формы должны быть входные данные

type Predictor() as this =
    inherit torch.nn.Module<torch.Tensor, torch.Tensor>("Predictor")
    
    let lstmDim = 50
    let hiddenSize = 100
    let numLayers = 5

    // Нет понимания как правильно построить слои в каком порядке на что опираться
    // Есть приблизительное понимание того что делает каждый слой

    let features = torch.nn.Sequential(
            torch.nn.Conv1d(100, 64, 5, 1),
            torch.nn.ReLU())
    // let encoder = torch.nn.LSTM(lstmDim, hiddenSize, numLayers)
    
    // let classifier = torch.nn.Sequential(
    //        torch.nn.Dropout(),
    //        torch.nn.Linear(hiddenSize, 4096),
    //        torch.nn.ReLU(),
    //        torch.nn.Dropout(),
    //        torch.nn.Linear(4096, 4096),
    //        torch.nn.ReLU(),
    //        torch.nn.Linear(4096, 7)
    //    )
         
    do
        this.RegisterComponents()
        
    override this.forward(input) =
        let x = features.forward(input)
        // let _, y, _ = encoder.forward(emb, Nullable()).ToTuple()
        // classifier.forward(y.squeeze(0)).argmax()
        x.argmax()

In [15]:
// Размер обучающего батча
let batchLength = 100

let trainTensors = 
        tensorData 
        |> Seq.map (fun (data, lable) -> 

            // Трансформируем свечи в тензоры
            let dataTensors = 
                data 
                |> Seq.map CandleQuote.toTensor
                |> Seq.toArray

            // Трансформируем лейбл в тензор
            let labelTensors = 
                lable 
                |> LabelType.toTensor

            (torch.cat(dataTensors), labelTensors))
        |> Seq.windowed batchLength
        |> Seq.map(fun x ->
            
            // Создаем батч

            let features = x |> Seq.map fst |> Seq.toArray
            let labels = x |> Seq.map snd |> Seq.toArray
            
            (torch.stack(features), torch.stack(labels)))
        |> Seq.toList

let model = new Predictor()
    
// Обучение модели (сейчас выдает ошибку)
model
|> TrainContext.getTrainContext
|> TrainContext.train trainTensors 10
|> Seq.iter (fun x -> printfn "%A" x)

Error: System.Runtime.InteropServices.ExternalException (0x80004005): Dimension out of range (expected to be in range of [-1, 0], but got 1)
Exception raised from maybe_wrap_dim_slow at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\WrapDimMinimal.cpp:31 (most recent call first):
00007FFFB51ED24200007FFFB51ED1E0 c10.dll!c10::Error::Error [<unknown file> @ <unknown line number>]
00007FFFB51B41E500007FFFB51B4170 c10.dll!c10::IndexError::IndexError [<unknown file> @ <unknown line number>]
00007FFFB51E06B500007FFFB51E0550 c10.dll!c10::detail::maybe_wrap_dim_slow<__int64> [<unknown file> @ <unknown line number>]
00007FFFB51E05B000007FFFB51E0550 c10.dll!c10::detail::maybe_wrap_dim_slow<__int64> [<unknown file> @ <unknown line number>]
00007FFFA6DD10DA00007FFFA6DD1080 torch_cpu.DLL!at::meta::structured__log_softmax::meta [<unknown file> @ <unknown line number>]
00007FFFA78A9A3300007FFFA78A4DD0 torch_cpu.DLL!at::cpu::where_outf [<unknown file> @ <unknown line number>]

https://towardsdatascience.com/encoder-decoder-model-for-multistep-time-series-forecasting-using-pytorch-5d54c6af6e60

https://towardsdatascience.com/stock-market-action-prediction-with-convnet-8689238feae3

https://cnvrg.io/pytorch-lstm/

https://towardsdatascience.com/time-series-forecasting-with-recurrent-neural-networks-74674e289816